# Lesson 3 – Solutions Notebook  
Sequential Pattern: Bug Report Cleaning & Triage Pipeline

This notebook contains **one possible solution** for the exercises in:

`03_sequential_bug_report_triage.ipynb`

Your own solutions may differ but still be valid as long as they implement the desired behavior.


## 1. Imports and Extended State Definition

We extend `BugState` to include:

- `is_third_party: bool` – whether the bug involves a plugin/extension (Stretch 1).
- `assigned_team: str` – which team should handle the bug (Stretch 2).


In [1]:
from typing import TypedDict

class BugState(TypedDict):
    raw_report: str
    clean_report: str
    severity: str
    platform: str
    feature: str
    recommendation: str
    is_third_party: bool
    assigned_team: str


## 2. Node 1 – Cleaning the Report (same as teaching notebook)

In [2]:
def clean_report_node(state: BugState) -> BugState:
    raw = state["raw_report"]
    clean = " ".join(raw.split())
    clean = clean.strip()
    state["clean_report"] = clean
    return state


## 3. Node 2 – Metadata Extraction (with extended severity + third-party flag)

This version implements:

- **Exercise 1 (More severity rules):**
  - If text contains `"data loss"` → `severity = "high"`.
  - Else if text contains `"typo"` and no high-severity keywords → `severity = "low"`.
- **Stretch Exercise 1:**
  - `is_third_party = True` if text contains `"plugin"` or `"extension"`.


In [3]:
def extract_metadata_node(state: BugState) -> BugState:
    text = state["clean_report"].lower()

    # high severity keywords
    high_keywords = ["urgent", "crash", "freez", "data loss"]

    if "data loss" in text:
        severity = "high"
    elif any(word in text for word in high_keywords):
        severity = "high"
    elif "typo" in text:
        severity = "low"
    else:
        severity = "medium"

    # platform
    if "mac" in text:
        platform = "mac"
    elif "windows" in text:
        platform = "windows"
    elif "web" in text or "browser" in text:
        platform = "web"
    else:
        platform = "unknown"

    # feature
    if "export" in text:
        feature = "export"
    elif "login" in text:
        feature = "login"
    else:
        feature = "unknown"

    # third-party flag
    if any(w in text for w in ["plugin", "extension"]):
        is_third_party = True
    else:
        is_third_party = False

    state["severity"] = severity
    state["platform"] = platform
    state["feature"] = feature
    state["is_third_party"] = is_third_party
    return state


## 4. Node 3 – Recommendation Generation (with unknown handling & third-party note)

This version implements:

- **Exercise 2 (Unknown handling):**
  - If `platform == "unknown"` or `feature == "unknown"`, append:
    > " Need more info on platform/feature."
- **Stretch Exercise 1:**
  - If `is_third_party` is `True`, append:
    > " Route to integrations team."


In [4]:
def make_recommendation_node(state: BugState) -> BugState:
    severity = state["severity"] or "unknown"
    platform = state["platform"] or "unknown"
    feature = state["feature"] or "unknown"
    is_third_party = state["is_third_party"]

    recommendation = f"{severity.title()} severity bug on {platform} in {feature}."

    if severity == "high":
        recommendation += " Assign to on-call engineer immediately."
    elif severity == "low":
        recommendation += " Can be handled as a low-priority UI issue."
    else:
        recommendation += " Add to regular sprint backlog."

    # Unknown handling
    if platform == "unknown" or feature == "unknown":
        recommendation += " Need more info on platform/feature."

    # Third-party note
    if is_third_party:
        recommendation += " Route to integrations team."

    state["recommendation"] = recommendation
    return state


## 5. Node 4 – Assign Team Node (Stretch Exercise 2)

Assigns a team based on `feature` (and could optionally consider `platform`).


In [5]:
def assign_team_node(state: BugState) -> BugState:
    feature = state["feature"]

    if feature == "export":
        team = "Data Export Team"
    elif feature == "login":
        team = "Auth Team"
    else:
        team = "General Backend Team"

    state["assigned_team"] = team
    return state


## 6. Wire Up the Sequential LangGraph Pipeline

We connect the nodes in this order:

1. `clean_report_node`
2. `extract_metadata_node`
3. `make_recommendation_node`
4. `assign_team_node`


In [6]:
from langgraph.graph import StateGraph

graph = StateGraph(BugState)

graph.add_node("clean_report", clean_report_node)
graph.add_node("extract_metadata", extract_metadata_node)
graph.add_node("make_recommendation", make_recommendation_node)
graph.add_node("assign_team", assign_team_node)

graph.set_entry_point("clean_report")

graph.add_edge("clean_report", "extract_metadata")
graph.add_edge("extract_metadata", "make_recommendation")
graph.add_edge("make_recommendation", "assign_team")

graph.set_finish_point("assign_team")

app = graph.compile()

## 7. Example Runs

Let's test a few different bug reports to see the pipeline in action.


In [7]:
# 7.1 High severity export bug on Mac with 'urgent'
state1: BugState = {
    "raw_report": "App keeps freezing when I click Export with CSV on Mac. Urgent!!!",
    "clean_report": "",
    "severity": "",
    "platform": "",
    "feature": "",
    "recommendation": "",
    "is_third_party": False,
    "assigned_team": "",
}
app.invoke(state1)

{'raw_report': 'App keeps freezing when I click Export with CSV on Mac. Urgent!!!',
 'clean_report': 'App keeps freezing when I click Export with CSV on Mac. Urgent!!!',
 'severity': 'high',
 'platform': 'mac',
 'feature': 'export',
 'recommendation': 'High severity bug on mac in export. Assign to on-call engineer immediately.',
 'is_third_party': False,
 'assigned_team': 'Data Export Team'}

In [8]:
# 7.2 Data loss mention → high severity
state2: BugState = {
    "raw_report": "Data loss when saving project using export plugin on Windows.",
    "clean_report": "",
    "severity": "",
    "platform": "",
    "feature": "",
    "recommendation": "",
    "is_third_party": False,
    "assigned_team": "",
}
app.invoke(state2)

{'raw_report': 'Data loss when saving project using export plugin on Windows.',
 'clean_report': 'Data loss when saving project using export plugin on Windows.',
 'severity': 'high',
 'platform': 'windows',
 'feature': 'export',
 'recommendation': 'High severity bug on windows in export. Assign to on-call engineer immediately. Route to integrations team.',
 'is_third_party': True,
 'assigned_team': 'Data Export Team'}

In [9]:
# 7.3 Low severity typo bug
state3: BugState = {
    "raw_report": "Small typo on settings page in the browser UI.",
    "clean_report": "",
    "severity": "",
    "platform": "",
    "feature": "",
    "recommendation": "",
    "is_third_party": False,
    "assigned_team": "",
}
app.invoke(state3)

{'raw_report': 'Small typo on settings page in the browser UI.',
 'clean_report': 'Small typo on settings page in the browser UI.',
 'severity': 'low',
 'platform': 'web',
 'feature': 'unknown',
 'recommendation': 'Low severity bug on web in unknown. Can be handled as a low-priority UI issue. Need more info on platform/feature.',
 'is_third_party': False,
 'assigned_team': 'General Backend Team'}

You can edit these test states or add more examples to deepen your understanding of the sequential pattern.

This notebook provides a clear, worked reference solution for **Lesson 3 – Sequential Pattern**.
